# Init

In [ ]:
# init
from importlib import reload
from pathlib import Path
import os
import sys
import pandas as pd

sys.path.append('..')
sys.path.append("C:/Users/gonta/D/software developer/packages")
import toolsGeneral.main as tgm
import toolsPandas.helpers as tph

def reloadPckg():
    reload(tgm)
    reload(tph)

# Import

In [2]:
# import
dataDir = Path(os.path.join(os.getcwd(), '..', 'data/normalized'))
rawFiles = [f for f in dataDir.rglob('*.pkl')]
print(len(rawFiles))

df_by_cntr = {
    file.parent.name:tgm.load(str(file))
    for file in rawFiles
}
print(len(df_by_cntr))

64
64


In [3]:
df_by_cntr['Algeria']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3
0,relation,192756,2,NaN,Algérie ⵍⵣⵣⴰⵢⴻⵔ الجزائر,NaN,DZ,NaN,NaN,NaN,NaN,NaN
1,relation,157062,4,NaN,Alger,NaN,NaN,DZ-16,NaN,NaN,NaN,NaN
2,relation,1258647,4,NaN,Béchar,NaN,NaN,DZ-08,NaN,NaN,NaN,NaN
3,relation,1258648,4,NaN,Naâma ⵏⵄⴰⵎⴰ النعامة,NaN,NaN,DZ-45,NaN,NaN,NaN,NaN
4,relation,1258649,4,NaN,El Bayadh,NaN,NaN,DZ-32,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2143,rel,1614560,8,2688117,Aïn El Assel ⵄⵉⵏ ⵍⴻⵄⵙⴻⵍ عين العسل,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2144,rel,1614578,8,2688117,El Tarf ⵟⵟⴰⵔⴻⴼ الطارف,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2145,rel,1616108,8,2688533,El Bouni ⵍⴱⵓⵏⵉ البوني,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2146,rel,1616094,8,2688534,El Hadjar ⵍⵃⴻⴵⴵⴰⵔ الحجار,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Apply fixes

## 1. osm basic fix

In [4]:
osm_basic_test = tgm.load(os.path.join('..', 'data', 'tests results', 'osm_basic_test.pkl'))
len(osm_basic_test)

64

In [22]:
tph.df_peek(tph.to_df(osm_basic_test).T)

,missing.name,leak
Albania,[],[]
Algeria,[],[]
Andorra,[],[]
Australia,[],[]
Austria,[],[52345]
Bangladesh,[14326679],"[3824588, 3825003, 3859335, 3921211, 3921222, 3921298, 3921322, 7318343, 9455570]"
Belarus,[],"[59162, 59189, 59195, 59275, 59506, 59752]"
Belgium,[],"[4217435, 7394, 7400, 7411, 7427, 7463, 7400]"
Bhutan,[],[]
Brazil,[],"[54882, 57963, 185579, 242620, 298204, 301079, 301464, 302635, 302819, 303702, 303781, 303940, 315173, 325866, 326266, 331463, 332476, 332924, 333597, 334051, 334443, 336819, 362413, 405935, 421151, 4059665, 298424, 298446]"


In [5]:
to_delete = []
for k,v in osm_basic_test.items():
    to_delete.extend(v['missing.name'])
    to_delete.extend(v['leak'])
len(to_delete)

167

In [6]:
# delete missing names and leaked elements
df_by_cntr_fixed = {}
for k,df in df_by_cntr.items():
    df_by_cntr_fixed[k] = df[~df['id'].isin(to_delete)].copy()

In [7]:
list(map(len,[df_by_cntr['Bangladesh'],df_by_cntr_fixed['Bangladesh']]))

[453, 443]

## 2. osm duplicates fix

In [9]:
data_dir = Path(os.path.join(os.getcwd(), '..', 'data/tests results/osm duplicates test'))

osm_dup_test = {
    file.parent.name:tgm.load(str(file))
    for file in data_dir.rglob('*/*.pkl')
}

In [29]:
to_delete_dup = []
for k,v in osm_dup_test.items():
    to_delete_dup.extend(v['duplicate.delete_elems'])
len(to_delete_dup)

158

In [89]:
# delete duplicate elements
for k,df in df_by_cntr_fixed.items():
    df_by_cntr_fixed[k] = df[~df[['id','tags.parent_id']].apply(tuple, axis=1).isin(to_delete_dup)].copy()

In [90]:
for k,df in df_by_cntr_fixed.items():
    path = os.path.join(os.getcwd(),f'../data/final/fixed/{k}/{k}_fixed.pkl')
    tgm.dump(path, df)

In [ ]:
list(map(len,[df_by_cntr_fixed['Bangladesh'], tempdict['Bangladesh']]))

[443, 441]

In [ ]:
l1 = df_by_cntr_fixed['Bangladesh'][['id','tags.parent_id']].apply(tuple, axis=1).to_list()
print(len(l1))
l2 = df_by_cntr_fixed['Bangladesh'][['id','tags.parent_id']].apply(tuple, axis=1).to_list()
print(len(l2))

443
441


In [86]:
[ele for ele in l1 if ele not in l2]

[('14326687', '3825003'), ('14326718', '3921322')]

In [5]:
dir = Path(os.path.join(os.getcwd(), '..', 'data/final/fixed'))
df_by_cntr_fixed = {
    file.parent.name:tgm.load(str(file))
    for file in [f for f in dir.rglob('*/*.pkl')]
}
len(df_by_cntr_fixed)

64

# Review

In [1]:
dir = Path(os.path.join(os.getcwd(), '..', 'data/fixed'))
df_by_cntr_fixed = {
    file.parent.name:tgm.load(str(file))
    for file in [f for f in dir.rglob('*/*.pkl')]
}
len(df_by_cntr_fixed)

64

In [4]:
df_by_cntr_fixed['Albania']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3
0,relation,53292,2,NaN,Shqipëria,NaN,AL,NaN,NaN,AL,NaN,NaN
1,relation,7739252,4,NaN,Shqipëria Jugore,NaN,NaN,NaN,NaN,AL03,AL03,NaN
2,relation,7739253,4,NaN,Shqipëria Qendrore,NaN,NaN,NaN,NaN,AL02,AL02,NaN
3,relation,7739254,4,NaN,Shqipëria Veriore,NaN,NaN,NaN,NaN,AL01,AL01,NaN
4,rel,1251469,6,7739252,Qarku i Fierit,NaN,NaN,AL-04,NaN,AL032,NaN,AL032
...,...,...,...,...,...,...,...,...,...,...,...,...
385,rel,1252600,8,1252589,Ersekë,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,rel,1252601,8,1252589,Maliq,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,rel,1252603,8,1252589,Progër,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,rel,1252604,8,1252589,Mollas,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def resume_divisions(rls : dict):
    resume = {}
    for k,df in rls.items():
        add_1 = df[df['tags.admin_level'] == '4']
        add_2 = df[df['tags.admin_level'] == '6']
        add_3 = df[df['tags.admin_level'] == '8']
        resume[k] = {'add_1': add_1, 'add_2': add_2, 'add_3': add_3}
    
    return pd.DataFrame(resume)

In [34]:
resume = resume_divisions(df_by_cntr_fixed)
len(resume)

3

In [25]:
df_by_cntr_fixed['Algeria']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3
0,relation,192756,2,NaN,Algérie ⵍⵣⵣⴰⵢⴻⵔ الجزائر,NaN,DZ,NaN,NaN,NaN,NaN,NaN
1,relation,157062,4,NaN,Alger,NaN,NaN,DZ-16,NaN,NaN,NaN,NaN
2,relation,1258647,4,NaN,Béchar,NaN,NaN,DZ-08,NaN,NaN,NaN,NaN
3,relation,1258648,4,NaN,Naâma ⵏⵄⴰⵎⴰ النعامة,NaN,NaN,DZ-45,NaN,NaN,NaN,NaN
4,relation,1258649,4,NaN,El Bayadh,NaN,NaN,DZ-32,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2143,rel,1614560,8,2688117,Aïn El Assel ⵄⵉⵏ ⵍⴻⵄⵙⴻⵍ عين العسل,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2144,rel,1614578,8,2688117,El Tarf ⵟⵟⴰⵔⴻⴼ الطارف,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2145,rel,1616108,8,2688533,El Bouni ⵍⴱⵓⵏⵉ البوني,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2146,rel,1616094,8,2688534,El Hadjar ⵍⵃⴻⴵⴵⴰⵔ الحجار,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
resume['Algeria']['add_2']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3
59,rel,4171631,6,1258650,Daïra Tsabit,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,rel,4171633,6,1258650,Daïra de Reggane,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,rel,4171636,6,1258650,Daïra d'Aoulef,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,rel,4171638,6,1258650,Daïra de Zaouiet Kounta,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,rel,4171639,6,1258650,Daïra Adrar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
601,rel,2688530,6,1455599,Daïra Aïn Berda,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,rel,2688531,6,1455599,Daïra Berrahal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,rel,2688532,6,1455599,Daïra Chetaibi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,rel,2688533,6,1455599,Daïra El Bouni,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
tph.df_peek(resume.T.map(len))

,add_1,add_2,add_3
Albania,3,12,374
Algeria,58,547,1541
Andorra,0,0,0
Australia,15,600,0
Austria,8,93,2075
Bangladesh,0,392,48
Belarus,1,128,1290
Belgium,3,10,4327
Bhutan,20,205,964
Brazil,3,44,711


In [ ]:
df_by_cntr_fixed